In [132]:
import re
import glob
from nltk.stem import PorterStemmer
from nltk.tokenize import WhitespaceTokenizer
import pickle
import numpy as np
import collections
import warnings 
warnings.filterwarnings('ignore')

In [9]:
def remove_special_characters(text):
    regex = re.compile('[^a-zA-Z0-9\s]')
    text_returned = re.sub(regex,' ',text)
    return text_returned

In [63]:
files = glob.glob('../Data files required/english-corpora/*.txt', recursive = True)
ps = PorterStemmer()
tk = WhitespaceTokenizer()
data = {}
i = 0
for file in files:
#     print(file,'\n')
    file1 = open(file,"r")
    text1 = file1.read()
    cleaned = remove_special_characters(text1)
    cleaned = cleaned.replace("\n"," ")
    cleaned = cleaned.replace("\t"," ")
    cleaned = re.sub('\s+',' ',cleaned)
    cleaned = cleaned.lower()
    token = tk.tokenize(cleaned)
    stemming_output = ' '.join([ps.stem(w) for w in token])
    name = file[16:-4]
    data[name] = stemming_output
#     print(data[name])
    i += 1
    if (i%300 == 0):
        print('Document no equals ',i)
#         break
    break # Comment this
# print(data['C00010'])

In [64]:
# with open('../Data files required/Processed_corpos_IR.pkl', 'wb') as f:
#     pickle.dump(data, f)

In [65]:
with open('../Data files required/Processed_corpos_IR.pkl', 'rb') as f:
    data = pickle.load(f)

In [74]:
index_dict = {}
i = 0
for file in data.keys():
    key = file
    index_dict[key] = i
    i += 1
    # if i == 5000:
    #     break
# print(index_dict)

In [78]:
index_dict2 = {v: k for k, v in index_dict.items()}
# len(index_dict2)

8351

#### Making a dict having info about no of words in each document.
#### Also calculating average no of words accross all documents :

In [80]:
No_W_doc_dict = {}
sum_ = 0
for doc in data.keys():
    No_W_doc_dict[doc] = len(data[doc])
    sum_ += len(data[doc])
Average_no_words = sum_ / 8351
    # print(doc)
    # break

## Making posting list : 

In [83]:
unique_words = []
unique_words_dict = {}
i = 0
for key in data.keys():
    i += 1
    for word in data[key].split():
        if unique_words_dict.get(word, 0)  == 0:
            unique_words.append(word)
            unique_words_dict[word] = 1
        else:
            pass
    if i%300 == 0:
        print('Document no ',i,' and unique words till now equals ',len(unique_words))
len(unique_words)

Document no  300  and unique words till now equals  55990
Document no  600  and unique words till now equals  90643
Document no  900  and unique words till now equals  120380
Document no  1200  and unique words till now equals  130917
Document no  1500  and unique words till now equals  137096
Document no  1800  and unique words till now equals  141646
Document no  2100  and unique words till now equals  152638
Document no  2400  and unique words till now equals  157973
Document no  2700  and unique words till now equals  160482
Document no  3000  and unique words till now equals  162144
Document no  3300  and unique words till now equals  186461
Document no  3600  and unique words till now equals  200096
Document no  3900  and unique words till now equals  212808
Document no  4200  and unique words till now equals  216325
Document no  4500  and unique words till now equals  217200
Document no  4800  and unique words till now equals  218310
Document no  5100  and unique words till now 

549255

In [85]:
# unique_words = []
# unique_words_dict = {}
temp_dict = {el:{} for el in unique_words}
# dict2 = {}
i = 0
for key in data.keys():
    i += 1
    for word in data[key].split():
        count = data[key].count(word)
        temp_dict[word][key] = count
        break # Comment this
    if i%300 == 0:
        print('Document completed till now equals ',i)
    break # Comment this


In [86]:
# with open('../Data files required/Saved_dictionary_IR.pkl', 'wb') as f:
#     pickle.dump(temp_dict, f)

In [87]:
with open('../Data files required/Saved_dictionary_IR.pkl', 'rb') as f:
    loaded_dict = pickle.load(f)

In [88]:
len(loaded_dict)

549255

### Making dict to fetch index no for every word and vice-versa

In [90]:
word_index = {}
i = 0
for word in unique_words:
    word_index[word] = i
    i += 1

# word_index

In [91]:
index_word = {v: k for k, v in word_index.items()}
# index_word

### Calculating IDF for every word and storing it :

In [93]:
word_IDF_dict = {}
for word in unique_words:
    word_IDF_dict[word] = np.log2((8351 + 1) / (len(loaded_dict[word].keys()) + 1))

### Making document norm required for TF-IDF model :

In [99]:
document_norm_dict = {}
for doc in data.keys():
    vector = np.zeros(len(unique_words), dtype = int)
    for word in unique_words:
        indx = word_index[word]
        try:
            tf = loaded_dict[word][doc]
        except:
            tf = 0
        vector[indx] = tf * word_IDF_dict[word]
        break   #  Comment this
    norm = np.linalg.norm(vector)
    document_norm_dict[doc] = norm
    break       #  Comment this

# document_norm_dict['S00602'][100000]

In [100]:
# with open('/content/drive/MyDrive/Colab Notebooks/document_norm_dict.pkl', 'wb') as f:
#     pickle.dump(document_norm_dict, f)

In [101]:
with open('../Data files required/document_norm_dict.pkl', 'rb') as f:
    document_norm_dict = pickle.load(f)

## Question 2

### (a) Boolean retrival model : 

In [106]:
def clean(query):
    regex = re.compile('[^a-zA-Z0-9\s]')
    cleaned = re.sub(regex,' ',query)
    cleaned = cleaned.replace("\n"," ")
    cleaned = cleaned.replace("\t"," ")
    cleaned = re.sub('\s+',' ',cleaned)
    cleaned = cleaned.lower()
    token = tk.tokenize(cleaned)
    processed_query = ' '.join([ps.stem(w) for w in token])
    return processed_query

In [107]:
query = input('Enter your query: ')
query = clean(query)
# print(query)
connecting_words = []
cnt = 1
different_words = []
for word in query.split():
    if word.lower() != "and" and word.lower() != "or" and word.lower() != "not":
        different_words.append(word.lower())
    else:
        connecting_words.append(word.lower())
print(different_words)
print(connecting_words)

Enter your query: python programming language
['python', 'program', 'languag']
[]


In [109]:
bool_dict = {}
# arr2 = np.zeros(3, dtype=bool)
for word in different_words:
    # print(word)
    if word in unique_words:
        i = 0
        bool_dict[word] = np.zeros(8351, dtype=bool)
        documents = list(loaded_dict[word].keys())
        indices = np.zeros(len(documents))
        for x in documents:
            try:
                indices[i] = index_dict[x]
                i += 1
            except:
                x = x[:-4]
                indices[i] = index_dict[x]
                i += 1
        for ind in indices:
            ind = int(ind)
            bool_dict[word][ind] = 1

# print(len(documents), len(indices))
print(bool_dict)
# bool_dict['cold'][40]

{'python': array([False, False, False, ..., False, False, False]), 'program': array([False, False,  True, ..., False,  True, False]), 'languag': array([False,  True, False, ..., False, False, False])}


In [110]:
query_list = query.split()
if len(query_list) == 1:
    value = query_list[0]
query_list

['python', 'program', 'languag']

#### Processing not in query : 

In [111]:
i = 0
for word in query_list:
    if word == 'not':
        next_word = query_list[i+1]
        query_list.remove(word)
        bool_dict[next_word] = ~ bool_dict[next_word]
    i += 1
print(query_list)
# bool_dict

['python', 'program', 'languag']


#### Processing 'and' in query :

In [112]:
i = 0
j = 1
while 'and' in  query_list:
    word = query_list[i]
    print(word)
    if word == 'and':
        prev_word = query_list[i-1]
        # print("i equals : ",i)
        next_word = query_list[i+1]
        # print('Prev and next equals : ',prev_word,' ',next_word)
        value = 'ans' + str(j)
        j += 1
        query_list[i] = value
        # print(query_list)
        query_list.remove(prev_word)
        # print("i equals : ",i)
        # print(query_list)
        query_list.remove(next_word)
        bool_dict[value] =  bool_dict[next_word] & bool_dict[prev_word]
        i = 0
    else :
        i += 1

print("Query list equals : ",query_list)
# print(bool_dict)
# print(j)

Query list equals :  ['python', 'program', 'languag']


#### Processing 'or' in query :

In [113]:
i = 0
while 'or' in  query_list:
    word = query_list[i]
    print(word)
    if word == 'or':
        prev_word = query_list[i-1]
        # print("i equals : ",i)
        next_word = query_list[i+1]
        # print('Prev and next equals : ',prev_word,' ',next_word)
        value = 'ans' + str(j)
        j += 1
        query_list[i] = value
        # print(query_list)
        query_list.remove(prev_word)
        # print("i equals : ",i)
        # print(query_list)
        query_list.remove(next_word)
        bool_dict[value] =  bool_dict[next_word] | bool_dict[prev_word]
        i = 0
    else :
        i += 1
    # if i >= len(query_list):
    #     i = 0

print("Query list equals : ",query_list)
# print(bool_dict)
# print(j)

Query list equals :  ['python', 'program', 'languag']


#### Processing other words : 

In [114]:
i = 0
while len(query_list) != 1:
    word = query_list[i]
    # print(word)
    prev_word = query_list[i]
    # print("i equals : ",i)
    next_word = query_list[i+1]
    # print('Prev and next equals : ',prev_word,' ',next_word)
    value = 'ans' + str(j)
    j += 1
    query_list[0] = value
    # query_list.remove(prev_word)
    # print("i equals : ",i)
    # print(query_list)
    query_list.remove(next_word)
    # print(query_list)
    
    bool_dict[value] =  bool_dict[next_word] & bool_dict[prev_word]
    # i = 0
    
print("Query list equals : ",query_list)
# print(bool_dict)
# print(j)

Query list equals :  ['ans2']


In [115]:
ans = bool_dict[value].copy()
indx = np.where(ans)[0]
final_ans = "Appropriate documents for given query are :  "
for x in indx:
    final_ans += index_dict2[x] + ', '
final_ans

'Appropriate documents for given query are :  S00371, S00001, S00162, S00029, S00070, S00735, S00346, S00888, M00046, M00067, M00108, M00180, M00184, M00294, M00304, T00673, C00610, C00712, C00259, C00406, C00417, C00062, C00560, C00677, C00227, C00046, C00137, C00778, C00217, C00780, C00721, C00383, C00787, C00078, C00269, C00063, C00047, C00722, C00303, C00129, C00056, C00360, C00727, C00767, C00319, C00807, C00275, C00543, C00523, C00474, C00390, C00860, C00115, C00022, C00325, C00014, C00163, C00180, C00598, C00187, C00197, C00028, C00986, C00149, C00013, C00328, C00725, C00533, C00231, C00374, C00932, C00743, C00247, C00459, C00703, C00473, C00337, C00018, C00005, C00763, C00688, C00468, C00737, C00338, C00664, C00393, C00074, C00084, C00002, C00851, C00096, C00704, C00410, C00723, C00914, C00272, C00609, C00920, C00250, C00918, C00425, C00790, C00570, C00795, C00021, C00298, C00132, C00616, C00940, C00007, C00817, C00454, C00370, C00103, C00511, C00071, C00536, C00010, C00302, C0

### (b) (20 marks) Implement a system from the Tf-Idf family with appropriate forms for the functions and tuned parameters. A query is matched using cosine similarity. : 

In [133]:
query = input('Enter your query: ')
query = clean(query)
# print(query)
query_list = query.split()
query_list

Enter your query: python and c++


['python', 'and', 'c']

In [135]:
query_vector = []
for word in query_list:

    if word in unique_words:
        idf_word = word_IDF_dict[word]
    else:
        idf_word = 0

    tf_idf = query_list.count(word) * idf_word
    query_vector.append(tf_idf)
#     q_norm+=tf_idf**2
# q_norm=math.sqrt(q_norm)

query_norm = np.linalg.norm(query_vector)
query_vector = np.array(query_vector) / query_norm
query_vector

array([0.97167289, 0.00417251, 0.236293  ])

In [137]:
# score = np.zeros(len(data.keys()))
score = {}
for i in index_dict2.keys():
    doc_vector = []
    for word in query_list:
        doc = index_dict2[i]
        if word in unique_words:
            idf_word = word_IDF_dict[word]
            try:
                tf = loaded_dict[word][doc]
            except:
                tf = 0
            tf_idf = tf * idf_word
        else:
            tf_idf = 0

        doc_vector.append(tf_idf)
    doc_vector = np.array(doc_vector) / document_norm_dict[doc]
    key = np.dot(query_vector,doc_vector)
    score[key] = doc

# score = sorted(score.items(),key=lambda x:x[1],reverse=True)

In [138]:
score_dict = collections.OrderedDict(sorted(score.items(),reverse=True))
# score_dict

In [139]:
# score_dict = collections.OrderedDict(sorted(score.items(),reverse=True))
# od.keys()
count = 1
for k,v in score_dict.items():
    print("Document ",count," equals ",v," ",k)
    count += 1
    if count == 11:
        break

Document  1  equals  C00005   0.18364900609049842
Document  2  equals  S00724   0.1602079458663031
Document  3  equals  S00785   0.15197272362818917
Document  4  equals  S00262   0.14380047125397788
Document  5  equals  S00791   0.13677040394364792
Document  6  equals  S00703   0.13388248956217375
Document  7  equals  C00962   0.13269226375929355
Document  8  equals  C00516   0.13111208218417256
Document  9  equals  C00730   0.12981945931284836
Document  10  equals  S00440   0.12866788673955343


### (c) (20 marks) Implement a system from the BM25 family with appropriate forms for the functions and tuned parameters.

#### Calculating IDF (for every word ) required for BM25 :

In [125]:
word_IDF_dict_BM25 = {}
for word in unique_words:
    freq = len(loaded_dict[word].keys())
    num = 8531 - freq + 0.5
    den = freq + 0.5
    word_IDF_dict_BM25[word] = np.log(1 + num/den)

##### BM25 started : 

In [127]:
query = input('Enter your query: ')
query = clean(query)
# print(query)
query_list = query.split()
query_list

Enter your query: python programming language


['python', 'program', 'languag']

In [128]:
score_BM25 = {}
k = 1.2
b = 0.75
for doc in data.keys():
    Sum = 0
    for word in query_list:
        try:
            tf = loaded_dict[word][doc]
            idf = word_IDF_dict_BM25[word]
            num = idf * tf * (k + 1)
            den = tf + k * (1 - b + b*No_W_doc_dict[doc]/Average_no_words )
            Sum += num/den
        except:
            pass
    score_BM25[Sum] = doc

In [130]:
score_dict_BM25 = collections.OrderedDict(sorted(score_BM25.items(),reverse=True))
print(score_dict_BM25)

OrderedDict([(12.3871565021086, 'C00005'), (12.01468016290079, 'C00892'), (11.932025316899862, 'C00712'), (11.859656934315488, 'C00237'), (11.822587488063023, 'C00669'), (11.79707857997897, 'C00197'), (11.764972611476507, 'C00462'), (11.684093072001238, 'C00044'), (11.590017549225454, 'C00425'), (11.582161005820069, 'C00753'), (11.579512906088402, 'C00543'), (11.542605505370869, 'C00962'), (11.464524494486836, 'C00737'), (11.4439807771987, 'C00338'), (11.43908341471, 'C00106'), (11.424037794587601, 'C00392'), (11.416595438155007, 'C00682'), (11.39854199301152, 'C00622'), (11.39415381529027, 'C00484'), (11.381144409800694, 'C00944'), (11.320626265532717, 'C00706'), (11.31094385909944, 'C00360'), (11.300377283657681, 'C00920'), (11.298202377232748, 'C00523'), (11.291467424737503, 'C00076'), (11.268039654538029, 'C00406'), (11.256707744950358, 'C00664'), (11.23260813268881, 'C00805'), (11.212914289498512, 'C00324'), (11.188030866116788, 'C00971'), (11.17212445193756, 'C00026'), (11.158723

In [131]:
count = 1
for k,v in score_dict_BM25.items():
    print("Document ",count," equals ",v," ",k)
    count += 1
    if count == 11:
        break

Document  1  equals  C00005   12.3871565021086
Document  2  equals  C00892   12.01468016290079
Document  3  equals  C00712   11.932025316899862
Document  4  equals  C00237   11.859656934315488
Document  5  equals  C00669   11.822587488063023
Document  6  equals  C00197   11.79707857997897
Document  7  equals  C00462   11.764972611476507
Document  8  equals  C00044   11.684093072001238
Document  9  equals  C00425   11.590017549225454
Document  10  equals  C00753   11.582161005820069


### Finished